# TAURUS + NSF REU 2021
# Introduction to Python Day 3
# Statistics, Interpolation & Fitting Data

Welcome back again! So far we have covered basic syntax, arrays, if statements, for loops, functions, and plotting. Now we can start building some data analysis skills using statistics. 

Python has lots of built in functions for generating random data. Today let's start with generating random values from a Gaussian distribution. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
gaussian = np.random.normal(0, 2, 1000) 
#Gaussian centered on 0 with a std. dev. of 2, sampled 1000 times

## Histograms 

A plot you'll probably make quite often is a histogram. For this, use plt.hist().

    plt.hist(data, bins, stacked=True/False, density=True/False)
    
The "bins" argument can either be an array, or simply setting "bins=x" for a number of evenly spaced bins. Setting stacked and density to True will normalize your data to sum to 1.

In [ ]:
plt.hist(gaussian, bins=20, stacked=True, density=True)
plt.grid(True)
plt.show()

Just like with normal plotting, there are lots of customizable options for histograms, including:

    facecolor: string argument giving color of histogram
    edgecolor: string argument giving edge color
    alpha: opacity, ranging from 0 to 1
    hatch: a histogram filling style, such as '\\', '//', '.', '..', etc
    histtype: bar, step
    density: True/False
    stacked: True/False
    cumulative: True/False
    
Full details here: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html
    
## Gaussian Fitting

Let's try fitting the Gaussian. This is a little bit redundant since we already know it's a Gaussian distribution, but this example should illustrate how the fitting process works. Let's use a package from scipy:

In [ ]:
from scipy.stats import norm

Pro-tip: if you're ever unsure about how to use a function, type ? after the function name to call up the docstrings.

To fit a Gaussian, simply use norm.fit(data). This will return the mean and standard deviation of the fit. np.average() and np.std() will give the mean and standard deviation of the actual data.

In [ ]:
mu, sigma = norm.fit(gaussian)
print(mu, sigma)

This did pretty well. It performs better the larger the sample you give it. 

Next, to overplot the Gaussian fit, you can calculate the probability density function (PDF).

In [ ]:
x = np.linspace(-5, 5, 1000)
pdf = norm.pdf(x, mu, sigma)

plt.plot(x, pdf, 'k-', linewidth=2)
plt.hist(gaussian, bins=50, color='mediumpurple', alpha=0.4, density=True, fill=True, histtype='step')
plt.title("Fit results: $\mu$ = %.2f, $\sigma$ = %.2f" % (mu, sigma)) 
#Pro-tip: this prints out the values of mu and sigma to 2 decimal places.

plt.show()

### Question 1: Sample the Poisson distribution centered on 4 with a sample size of your choice. Plot the histogram. Hint: you can check the documentation for np.random, or try tab-complete to find the correct function.

In [ ]:
# solution here

## Interpolating Data

You may encounter a situation in your research where you will need to interpolate a model to interpret your data. The scipy library has many different interpolating (and extrapolating) algorithms, but the simplest is a 1D interpolation, called interp1d. 

Let's first generate a well-behaved function that will be easy to interpolate along.

In [ ]:
xarray = np.linspace(-4*np.pi, 4*np.pi, 30)
yarray = np.sin(xarray)

plt.plot(xarray, yarray, 'o')
plt.show()

What do you notice about this plot? Well, at first glance it doesn't really look like a sine curve because it is poorly sampled. Of course, since we already know it's a sine curve, we could just sample the function better, but say you didn't have such a nice function. Interp1d comes to the rescue! 

In [ ]:
from scipy.interpolate import interp1d

There are a few steps to interpolation. First, the interp1d function interpolates your original y array. Then, you need to define the new array along which to interpolate, which must completely fit inside the original array (or else it's extrapolation). We'll take the same array but grid it more finely. Finally, you create a new y array using the interpolated function.

In [ ]:
interp = interp1d(xarray, yarray)
xnew = np.linspace(-4*np.pi, 4*np.pi, 1000)
ynew = interp(xnew)

plt.plot(xnew, ynew, '--')
plt.plot(xarray, yarray, 'or')
plt.show()

Now it looks much closer to a sine curve!

## Loading data files into Python 

Up until now, none of the exercises have required you to load in an outside file to the notebook, but this is definitely something you'll need to learn! You can use np.genfromtxt, np.loadtxt, or a library called pandas if you're feeling advanced. 

If your file is just a 1D array of floats, then simply defining a variable with np.genfromtxt('filename') will suffice. Other arguments include "dtype" which allows you to specify that they're strings or integers, and in a moment I'll show you how to transpose a text file.

First let's load up the spectrum we are going to work with. Python reads by rows, not columns, but if you want separate arrays indicating the columns such as wavelengths, flux, and flux errors, you can transpose it, by doing np.genfromtxt('filename').T. You can define it all in one line, like so:

x, y, z = np.genfromtxt('file_with_three_columns.txt').T

### Question 1: Load up the file "test_spectrum.txt", whose three columns are wavelength, flux, and error.

In [ ]:
# load file here

## Fitting a Spectrum

Finally we can put what we've learned together to do a real life astronomy example! We will do this exercise together.

Here is a sample spectrum which features a prominent spectral emission line. Our job will be to fit the line with a Gaussian model to compute its peak flux and FWHM. 


Let's plot up what we have so far. This is a great opportunity to show you another useful plotting tool: errorbars! 

Instead of doing plt.plot, you can do plt.errorbar, which plots both the data and the errorbars. The first two arguments are x and y like in plt.plot, and the next two arguments are xerr and yerr. You can specify the color of the errorbars using the keyword argument (kwarg) "ecolor", and you can put hats on the errorbar with "capsize."

### Question 2: Plot the spectrum with y errors (it doesn't have x errors) with the data in blue and the errorbars in magenta.

In [ ]:
# your plot here

We're going to use the optimize function from scipy. This allows you to fit pretty much any kind of function you want, but here we know that the emission line has a Gaussian profile. 

Many spectral lines are well-described by a Gaussian curve, and lines that deviate from a Gaussian are often astronomically interesting. So the first step in many spectroscopic projects is to see if your line is Gaussian.

This means that $\mu$, the mean of the Gaussian, is the wavelength at which the line peaks. This helps you decide what line it is (Ly$\alpha$, carbon monoxide, nitrogen, etc.)

$\sigma$ is the width of the line -- spectral lines have natural shifts a bit to the left and a bit to the right of the "true" wavelength it should peak at. There are astronomical reasons why your line would be extremely wide, or skewed to one side.

f0 is the peak height of the line, or the flux at which your line peaks -- how bright your astronomical object is. The reason we call it "f0" instead of the full Gaussian term is we don't want it to get caught up in the value of $\sigma$.

### Question 3: Import the optimize function from scipy as "opt." Using what you learned yesterday, define a Gaussian function which takes as its arguments x, $f_0$, $\mu$, and $\sigma$. (Here, $f_0$ is a normalizing factor in front of the Normal distribution, used in place of the $(\sigma * \sqrt{2 \pi})^{-1}$ term.) When you use optimize, it's important that the first argument is the array you're fitting along, so here that is x. 

In [ ]:
# import your function here

# define your gaussian here (hint: if you did your homework you did this already!)

The optimize function finds the best fit parameters as well as the covariance matrix for those parameters. p0 is the initial guess for the fit - this is important because a bad or no first guess will cause a bad fit. So the rest of the fit goes like so:

See if you can guess a good p0!

*Make sure the order of p0 is the same as the order of the arguments in the function!*

In [ ]:
bfpars, covar = opt.curve_fit(gauss, wave, flux, p0=[10700, 100, 2.0e-18], sigma=error) #edit p0

print(bfpars)
#so now we see the line peaks at 10,704 Angstrom, it has a width of 219 Angstrom, 
#and it has a peak flux of 1.88e-18 erg s^-1 cm^-2 AA^-1

"Bfpars" is now an array containing the best fit parameters, in the same order as your function. So in my case, bfpars[0] is $\mu$, bfpars[1] is $\sigma$, and bfpars[2] is f0.

Try printing out the best fit parameters. Do the values make sense to you? 

Let's define a higher resolution wavelength grid we can use to overlay the fit. We can also zoom in on the line since the rest of the spectrum is noise.

Below, x should be a grid that is centered on $\mu$, and is plotted from -10$\sigma$ to 10$\sigma$ (so make sure your index values are correct, depending on how you defined the Gaussian function -- mine has $\mu$, $\sigma$, f0 in that order).  This just means I want you to evaluate the Gaussian right where the emission line occurs.

In [ ]:
x = np.linspace(bfpars[0]-10*bfpars[1], bfpars[0]+10*bfpars[1], 1000) 
#Center the grid on the wavelength of the line
#and go +- 10*sigma from there

modelfit = gauss(x, bfpars[0], bfpars[1], bfpars[2]) 
#This is the Gaussian curve with the best fit parameters

## Saving Data Files

The syntax for writing files is similar to loading them, with many customizable options in np.savetxt(filename, data). The arguments are as follows:

    fname: the path and filename you want to save to
    X: the array(s) being saved
    fmt: the format of each column of data, e.g. "%s" for strings or "%.2f" for a floating point with 2 decimal places
    delimiter: can leave this blank for evenly spaced columns, but you could use ',' for comma-separated values
    header: a string that might contain info about the file or the names of columns, like '# ra dec velocity'
    
    
Save the modelfit parameters to a text file in this directory.
    

In [ ]:
# save file here

### Question 3: As your final task, plot the model over the data by including both plots in the same cell, and make sure you zoom in on the line by choosing a wise range for xlim. 

Spectra are typically viewed along a long x-axis: try changing the figure size.

Congrats, you've done your first bit of science with Python!

In [ ]:
# solution here